In [ ]:
# Import modules
from ax.api.client import Client
from ax.api.configs import  RangeParameterConfig, ChoiceParameterConfig
import json


In [ ]:
# TODO modify config.json file to use Ax conventions for parameters
# TODO make the following functions
    # 1. read in excel spreadsheet and translate to JSON with variable names
    # 2. read in JSON and produce Ax parameters


In [ ]:
# read in the config file and set up parameters

# Bounds and configs are loaded in config file (config.json)
configPath = 'config.json'

# Load the full config data file
try:
    # Load the full config data file
    with open(configPath, 'r') as f:
        config = json.load(f)
        print(f"Loaded config: file from {configPath}")

    # Extract bounds for parameters
    pbounds = config['pbounds']

    ## Other parameters that can be loaded via config
    # optimizer_settings = config['optimizer_settings']
    # model_params = config['model_parameters']

    # Print usage
    print("Loaded config parameters:")
    print(json.dumps(config, indent=4))


except FileNotFoundError:
    print(f"Error: Could not find the config file at '{configPath}'. Please check the path and try again.")

except json.JSONDecodeError as e:
    print(f"Error: Failed to parse JSON in config file '{configPath}': {e}")

except Exception as e:
    print(f"An unexpected error occurred while loading the config file: {e}")



In [ ]:
infill_density = RangeParameterConfig(name="infill_density", parameter_type="float", bounds=(0,100))
layer_height = RangeParameterConfig(name="layer_height", parameter_type="float", bounds=(0.1,0.4))
infill_type = ChoiceParameterConfig(name="infill_type", parameter_type="str", values=["honeycomb", "gyroid", "lines", "rectilinear"])

client.configure_experiment(
    parameters=[infill_density, layer_height, infill_type],
    # the following areguments are only necessary when saving to the DB
    name="3D_print_strength_experiment",
    description="Maximize the strength of the 3D printed parts",
    owner="developer",
    )